In [4]:
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
from multiprocessing import Pool
import datetime

In [2]:
train=pd.read_csv('../train.csv')
test=pd.read_csv('../test_QyjYwdj.csv')
item=pd.read_csv('../item_data.csv')
transaction=pd.read_csv('../customer_transaction_data.csv')
coupon_item_mapping=pd.read_csv('../coupon_item_mapping.csv')
camp=pd.read_csv('../campaign_data.csv')
camp.start_date=pd.to_datetime(camp.start_date,format='%d/%m/%y')
camp.end_date=pd.to_datetime(camp.end_date,format='%d/%m/%y')
camp.sort_values('start_date',inplace=True)
df=pd.concat((train.iloc[:,:-1],test),axis=0)
df=df.merge(camp,on='campaign_id',how='left')
transaction.date=pd.to_datetime(transaction.date)

In [5]:
temp=df[['customer_id','coupon_id','start_date','id','end_date']]
temp['cut_off_date']=pd.to_datetime(df.start_date) -datetime.timedelta(150)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
transaction=transaction.drop_duplicates()

In [7]:
transaction.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [8]:
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id==data_['coupon_id']].item_id))\
               &(transaction.date<data_['start_date'])]    
    
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [9]:
%%time
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

CPU times: user 20.4 s, sys: 572 ms, total: 21 s
Wall time: 11min 29s


In [10]:
cols=['id','num_bought_coup','num_discount_availed_coup','frac_discount_availed_coup','sum_discount_coup',
'selling_price_sum_coup',
'coupon_discount_mean_coup','selling_price_mean_coup','other_discount_sum_coup',
      'other_discount_mean_coup','cd_sp_ratio_mean_coup','cd_sp_ratio_sum_coup'
     ]
# cols=['id','num_bought_coup_end_date','num_discount_availed_coup_end_date','frac_discount_availed_coup_end_date',
#       'sum_discount_coup_end_date','selling_price_sum_coup_end_date']
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [11]:
feat_df.head()

,id,num_bought_coup,num_discount_availed_coup,frac_discount_availed_coup,sum_discount_coup,selling_price_sum_coup,coupon_discount_mean_coup,selling_price_mean_coup,other_discount_sum_coup,other_discount_mean_coup,cd_sp_ratio_mean_coup,cd_sp_ratio_sum_coup
0,1.0,3309.0,158.0,0.047749,-4662.46,286278.21,-1.409024,86.515023,-207715.95,-62.773028,-0.020780,-68.762572
1,2.0,66.0,0.0,0.000000,0.00,4717.27,0.000000,71.473788,-677.15,-10.259848,0.000000,0.000000
2,6.0,99.0,13.0,0.131313,-463.06,32844.59,-4.677374,331.763535,-14974.90,-151.261616,-0.014480,-1.433481
3,7.0,38.0,1.0,0.026316,-17.81,4076.41,-0.468684,107.273947,-302.79,-7.968158,-0.006008,-0.228304
4,9.0,1060.0,12.0,0.011321,-231.53,202231.56,-0.218425,190.784491,-44172.62,-41.672283,-0.001419,-1.504496


In [12]:
feat_df.to_csv('../coupon_item_feats_no_lag_v2.csv',index=False)

In [13]:
def get_feat(row):
    data_=row[1]
#     temp=transaction[(transaction.customer_id==data_['customer_id'])&\
#                          (transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id==data_['coupon_id']].item_id))\
#                &(transaction.date<data_['start_date'])]

    temp=transaction[(transaction.customer_id==data_['customer_id'])&\
                         (transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id==data_['coupon_id']].item_id))\
               &(transaction.date<data_['start_date'])]
    
    
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [14]:
# cols=['id','num_bought','num_discount_availed','frac_discount_availed','sum_discount','selling_price_sum']
cols=['id','num_bought','num_discount_availed','frac_discount_availed','sum_discount',
'selling_price_sum',
'coupon_discount_mean','selling_price_mean','other_discount_sum',
      'other_discount_mean','cd_sp_ratio_mean','cd_sp_ratio_sum'
     ]

In [15]:
%%time
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

CPU times: user 20.5 s, sys: 648 ms, total: 21.2 s
Wall time: 11min 48s


In [16]:
len(data_outputs)

128595

In [17]:
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [18]:
feat_df.to_csv('../feats_user_coup_item_no_lag_v2.csv',index=False)

In [19]:
camp_cust=df.groupby(['campaign_id','customer_id'])['coupon_id'].unique().reset_index()

In [20]:
camp_cust=camp_cust.merge(camp,on='campaign_id')

In [21]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_['customer_id'])&\
                             (transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id.isin(data_['coupon_id'])].item_id))\
                   &(transaction.date<data_['start_date'])]

    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, camp_cust.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_camp_cust','num_discount_availed_camp_cust','frac_discount_availed_camp_cust','sum_discount_camp_cust',
'selling_price_sum_camp_cust',
'coupon_discount_mean_camp_cust','selling_price_mean_camp_cust','other_discount_sum_camp_cust',
      'other_discount_mean_camp_cust','cd_sp_ratio_mean_camp_cust','cd_sp_ratio_sum_camp_cust'
     ]

camp_cust_hist=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

CPU times: user 1.12 s, sys: 276 ms, total: 1.39 s
Wall time: 4min 29s


In [22]:
camp_cust_hist.to_csv('../camp_cust_feats_v2.csv',index=False)

In [23]:
a=pd.read_csv('../feats_user_coup_item_no_lag_v2.csv')

In [24]:
b=pd.read_csv('../../av_amex/feats_user_coup_item_no_lag_v2.csv')